In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1691410809424_0003,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('trusted_bureau_20230805').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_AULA_ = '20230804'
_REF_ = '202001'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
raw ='t_bureau_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
bucket_trusted = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/003_trusted'.format(dia=_DIA_AULA_)
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_AULA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_AULA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
output_trusted = 'trusted_bureau_20230805'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
full_path_raw = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_raw)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_bureau_20230805

In [10]:
path_lake = os.path.join(full_path_raw)
raw_00=sqlContext.read.format("parquet").load(path_lake)
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
raw_00.show(5,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------------------+--------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+-------------+----------------------------+-----------------+
|ref   |ts_file_generation|ts_proc       |SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|ref_partition|ts_file_generation_partition|ts_proc_partition|
+------+------------------+--------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+---

In [24]:
# +------+------------------+--------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+-------------+----------------------------+-----------------+
# |ref   |ts_file_generation|ts_proc       |SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|ref_partition|ts_file_generation_partition|ts_proc_partition|
# +------+------------------+--------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+-------------+----------------------------+-----------------+
# |202308|2023080500        |20230806155119|215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0           |null                  |0                 |91323.0       |0.0                |null                |0.0                   |Consumer credit|-131              |null       |202308       |2023080500                  |20230806155119   |
# |202308|2023080500        |20230806155119|215354    |5714463     |Active       |currency 1     |-208       |0                 |1075.0             |null             |null                  |0                 |225000.0      |171342.0           |null                |0.0                   |Credit card    |-20               |null       |202308       |2023080500                  |20230806155119   |
# |202308|2023080500        |20230806155119|215354    |5714464     |Active       |currency 1     |-203       |0                 |528.0              |null             |null                  |0                 |464323.5      |null               |null                |0.0                   |Consumer credit|-16               |null       |202308       |2023080500                  |20230806155119   |
# |202308|2023080500        |20230806155119|215354    |5714465     |Active       |currency 1     |-203       |0                 |null               |null             |null                  |0                 |90000.0       |null               |null                |0.0                   |Credit card    |-16               |null       |202308       |2023080500                  |20230806155119   |
# |202308|2023080500        |20230806155119|215354    |5714466     |Active       |currency 1     |-629       |0                 |1197.0             |null             |77674.5               |0                 |2700000.0     |null               |null                |0.0                   |Consumer credit|-21               |null       |202308       |2023080500                  |20230806155119   |
# +------+------------------+--------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+-------------+----------------------------+-----------------+
# only showing top 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
print('lista de colunas para tipar')
for col in raw_00.columns:
    print('cast(' + col + ' as) as ' + col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lista de colunas para tipar
cast(ref as) as ref,
cast(ts_file_generation as) as ts_file_generation,
cast(ts_proc as) as ts_proc,
cast(SK_ID_CURR as) as SK_ID_CURR,
cast(SK_ID_BUREAU as) as SK_ID_BUREAU,
cast(CREDIT_ACTIVE as) as CREDIT_ACTIVE,
cast(CREDIT_CURRENCY as) as CREDIT_CURRENCY,
cast(DAYS_CREDIT as) as DAYS_CREDIT,
cast(CREDIT_DAY_OVERDUE as) as CREDIT_DAY_OVERDUE,
cast(DAYS_CREDIT_ENDDATE as) as DAYS_CREDIT_ENDDATE,
cast(DAYS_ENDDATE_FACT as) as DAYS_ENDDATE_FACT,
cast(AMT_CREDIT_MAX_OVERDUE as) as AMT_CREDIT_MAX_OVERDUE,
cast(CNT_CREDIT_PROLONG as) as CNT_CREDIT_PROLONG,
cast(AMT_CREDIT_SUM as) as AMT_CREDIT_SUM,
cast(AMT_CREDIT_SUM_DEBT as) as AMT_CREDIT_SUM_DEBT,
cast(AMT_CREDIT_SUM_LIMIT as) as AMT_CREDIT_SUM_LIMIT,
cast(AMT_CREDIT_SUM_OVERDUE as) as AMT_CREDIT_SUM_OVERDUE,
cast(CREDIT_TYPE as) as CREDIT_TYPE,
cast(DAYS_CREDIT_UPDATE as) as DAYS_CREDIT_UPDATE,
cast(AMT_ANNUITY as) as AMT_ANNUITY,
cast(ref_partition as) as ref_partition,
cast(ts_file_generation_partition a

In [14]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            ref,
            ref_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            cast(SK_ID_CURR as int) as SK_ID_CURR,
            cast(SK_ID_BUREAU as int) as SK_ID_BUREAU,
            cast(CREDIT_ACTIVE as string) as CREDIT_ACTIVE,
            cast(CREDIT_CURRENCY as string) as CREDIT_CURRENCY,
            cast(DAYS_CREDIT as int) as DAYS_CREDIT,
            cast(CREDIT_DAY_OVERDUE as int) as CREDIT_DAY_OVERDUE,
            cast(DAYS_CREDIT_ENDDATE as double) as DAYS_CREDIT_ENDDATE,
            cast(DAYS_ENDDATE_FACT as double) as DAYS_ENDDATE_FACT,
            cast(AMT_CREDIT_MAX_OVERDUE as double) as AMT_CREDIT_MAX_OVERDUE,
            cast(CNT_CREDIT_PROLONG as int) as CNT_CREDIT_PROLONG,
            cast(AMT_CREDIT_SUM as double) as AMT_CREDIT_SUM,
            cast(AMT_CREDIT_SUM_DEBT as double) as AMT_CREDIT_SUM_DEBT,
            cast(AMT_CREDIT_SUM_LIMIT as double) as AMT_CREDIT_SUM_LIMIT,
            cast(AMT_CREDIT_SUM_OVERDUE as double) as AMT_CREDIT_SUM_OVERDUE,
            cast(CREDIT_TYPE as string) as CREDIT_TYPE,
            cast(DAYS_CREDIT_UPDATE as int) as DAYS_CREDIT_UPDATE,
            cast(AMT_ANNUITY as double) as AMT_ANNUITY
            
        from
            raw_00
        --where
            --ref_partition = {_ref_}
        --    ref = {_ref_}
            
            
    """.format(pdthproc=dthproc,_ref_ = _REF_))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1716428

In [15]:
lake.explain()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

== Physical Plan ==
InMemoryTableScan [ref#0, ref_partition#20, ts_proc#116L, ts_proc_partition#117L, SK_ID_CURR#118, SK_ID_BUREAU#119, CREDIT_ACTIVE#120, CREDIT_CURRENCY#121, DAYS_CREDIT#122, CREDIT_DAY_OVERDUE#123, DAYS_CREDIT_ENDDATE#124, DAYS_ENDDATE_FACT#125, AMT_CREDIT_MAX_OVERDUE#126, CNT_CREDIT_PROLONG#127, AMT_CREDIT_SUM#128, AMT_CREDIT_SUM_DEBT#129, AMT_CREDIT_SUM_LIMIT#130, AMT_CREDIT_SUM_OVERDUE#131, CREDIT_TYPE#132, DAYS_CREDIT_UPDATE#133, AMT_ANNUITY#134]
   +- InMemoryRelation [ref#0, ref_partition#20, ts_proc#116L, ts_proc_partition#117L, SK_ID_CURR#118, SK_ID_BUREAU#119, CREDIT_ACTIVE#120, CREDIT_CURRENCY#121, DAYS_CREDIT#122, CREDIT_DAY_OVERDUE#123, DAYS_CREDIT_ENDDATE#124, DAYS_ENDDATE_FACT#125, AMT_CREDIT_MAX_OVERDUE#126, CNT_CREDIT_PROLONG#127, AMT_CREDIT_SUM#128, AMT_CREDIT_SUM_DEBT#129, AMT_CREDIT_SUM_LIMIT#130, AMT_CREDIT_SUM_OVERDUE#131, CREDIT_TYPE#132, DAYS_CREDIT_UPDATE#133, AMT_ANNUITY#134], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *

In [16]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: integer (nullable = true)
 |-- CREDIT_DAY_OVERDUE: integer (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: double (nullable = true)
 |-- DAYS_ENDDATE_FACT: double (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: double (nullable = true)
 |-- CNT_CREDIT_PROLONG: integer (nullable = true)
 |-- AMT_CREDIT_SUM: double (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: double (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: double (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: double (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: integer (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)

In [17]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|ref   |ref_partition|ts_proc       |ts_proc_partition|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+------+-------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+-------

In [25]:
# +------+-------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |ref   |ref_partition|ts_proc       |ts_proc_partition|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
# +------+-------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# |202308|202308       |20230807094525|20230807094525   |215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0           |null                  |0                 |91323.0       |0.0                |null                |0.0                   |Consumer credit|-131              |null       |
# |202308|202308       |20230807094525|20230807094525   |215354    |5714463     |Active       |currency 1     |-208       |0                 |1075.0             |null             |null                  |0                 |225000.0      |171342.0           |null                |0.0                   |Credit card    |-20               |null       |
# |202308|202308       |20230807094525|20230807094525   |215354    |5714464     |Active       |currency 1     |-203       |0                 |528.0              |null             |null                  |0                 |464323.5      |null               |null                |0.0                   |Consumer credit|-16               |null       |
# |202308|202308       |20230807094525|20230807094525   |215354    |5714465     |Active       |currency 1     |-203       |0                 |null               |null             |null                  |0                 |90000.0       |null               |null                |0.0                   |Credit card    |-16               |null       |
# +------+-------------+--------------+-----------------+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
# only showing top 4 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
path_trusted = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted)

lake.coalesce(1).write. \
partitionBy("ref_partition","ts_proc_partition"). \
parquet(path_trusted,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/003_trusted/trusted_bureau_20230805

In [19]:
varDataFile = output_trusted

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6
            order by
                1,2,3,4,5,6
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+
|name_file              |name_file_partition    |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+
|trusted_bureau_20230805|trusted_bureau_20230805|202308|202308       |20230807094525|20230807094525   |1716428      |
+-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+

In [26]:
# +-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+
# |name_file              |name_file_partition    |ref   |ref_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+
# |trusted_bureau_20230805|trusted_bureau_20230805|202308|202308       |20230807094525|20230807094525   |1716428      |
# +-----------------------+-----------------------+------+-------------+--------------+-----------------+-------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
path_control = os.path.join('s3://',bucket_control,'tb_0002_controle_procesamento_trusted')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0002_controle_procesamento_trusted